In [1]:
from ML_Preparation.Preprocessing_M import DataExtractor, DataPreprocessing

de = DataExtractor("D:/hka-aqm-m", "C:/Users/danie/building_M_data")

df = de.create_df()

No .dat files found in C:/Users/danie/building_M_data. 
 Trying to extract files from the original directory D:/hka-aqm-m
Read data successfully.
Data contains 979166 data points and 18 columns.


In [2]:
dp = DataPreprocessing()

df_preprocessed = dp.preprocess_df(df, rolling_window = "7d", sample_time = "1d")

In [8]:
df_preprocessed.columns

Index(['date_time', 'tmp', 'hum', 'CO2', 'VOC', 'vis', 'IR', 'BLE', 'rssi',
       'snr', 'tavg', 'tmin', 'tmax', 'prcp', 'wdir', 'wspd', 'wpgt', 'pres',
       'room_number', 'time_diff_sec', 'hum_diff', 'CO2_diff', 'VOC_diff',
       'vis_diff', 'IR_diff', 'BLE_diff', 'tavg_diff', 'tmin_diff',
       'tmax_diff', 'prcp_diff', 'wdir_diff', 'wspd_diff', 'wpgt_diff',
       'pres_diff', 'year', 'dayofweek', 'hour', 'season', 'color',
       'VOC_CO2_ratio'],
      dtype='object')

In [2]:
# df_preprocessed.to_parquet("hka_M_preprocessed.parquet")

In [4]:
# import pandas as pd
# df_preprocessed = pd.read_parquet("hka_M_preprocessed.parquet")

In [5]:
# from Feature_Engineering import *
# fe = FeatureEngineering(df_preprocessed.drop(columns = ["VOC_diff", "VOC_diff_per_sec", "VOC_CO2_ratio"]), label = "CO2", categorical_features = ["season", "room_number", "dayofweek", "color"])

In [9]:
from ML_Preparation.Feature_Engineering import *
fe = FeatureEngineering(df_preprocessed.drop(columns = ["VOC_CO2_ratio"]), label = "CO2", categorical_features = ["season", "room_number", "dayofweek", "color"])

KeyError: "['CO2_diff_per_sec'] not found in axis"

In [4]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(fe.X_train, fe.y_train)

LinearRegression()

In [5]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error

pred_test = lr.predict(fe.X_test)

pred_train = lr.predict(fe.X_train)

print(f"Training score: {mean_absolute_percentage_error(y_true = fe.y_train, y_pred = pred_train)}")

print(f"Training score: {mean_absolute_percentage_error(y_true = fe.y_test, y_pred = pred_test)}")

Training score: 0.1266962718670971
Training score: 2817528474.003649


In [9]:
fe.X_train.shape

(5680, 84)

In [10]:
time_steps = 7
def create_ds(data, t_steps):
    data = pd.DataFrame(data)
    data_s = data.copy()
    for i in range(time_steps):
        data = pd.concat([data, data_s.shift(-(i+1))], axis = 1)   
    data.dropna(axis=0, inplace=True)
    return data.values

X_train_stepped = create_ds(fe.X_train, 7)

In [11]:
y_train_stepped = create_ds(fe.y_train, 7)

In [12]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Dropout, Bidirectional
from keras.optimizers import Adam, SGD

class MyLSTMModel:
    def __init__(self, lstm_units=int(fe.X_train.shape[1]),input_shape=(None, fe.X_train.shape[1])):
        self.model = Sequential()
        self.model.add(LSTM(units=lstm_units, input_shape=input_shape, return_sequences = True, recurrent_dropout = 0.3))
        self.model.add(Dropout(0.3))
        self.model.add(LSTM(units=lstm_units*2, return_sequences = True, recurrent_dropout = 0.3))
        self.model.add(Dropout(0.3))
        self.model.add(Bidirectional(LSTM(units=lstm_units, recurrent_dropout = 0.3)))
        self.model.add(Dense(60, activation = "relu"))
        self.model.add(Dense(40, activation = "relu"))
        self.model.add(Dense(20, activation = "relu"))

        self.model.add(Dense(1))

        self.model.compile(optimizer=Adam(learning_rate = 0.05), loss='mse')
    
    def summary(self):
        self.model.summary()

# Instantiate the model
my_model = MyLSTMModel()

# Print the model summary
X_train_array = fe.X_train.values.reshape(fe.X_train.shape[0], 1, fe.X_train.shape[1]).astype(np.float32)
y_train_array = fe.y_train.values.astype(np.float32).reshape(fe.y_train.shape[0], 1, 1)
my_model.model.fit(X_train_array, y_train_array, epochs = 15, batch_size = 200)

c:\Users\danie\anaconda3\envs\awp1\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 210765.1250
Epoch 2/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 40727.1641
Epoch 3/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 31430.5820
Epoch 4/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 31372.5312
Epoch 5/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 32380.7793
Epoch 6/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 31377.6348
Epoch 7/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 30940.4668
Epoch 8/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 31167.1582
Epoch 9/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 32148.4668
Epoch 10/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 32656.2793
Epoch 11/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 22557.9199
Epoch 12/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9372.2021
Epoch 13/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9111.5977
Epoch 14/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7856.4834
Epoch 15/15
29/29 ━━━━━━━━━━━━━

In [13]:
X_test_array = fe.X_test.values.reshape(fe.X_test.shape[0], 1, fe.X_test.shape[1]).astype(np.float32)
y_test_array = fe.y_test.values.astype(np.float32).reshape(fe.y_test.shape[0], 1, 1)
pred = my_model.model.predict(X_test_array)

45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step 


In [14]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

def evaluate_model(y_true, y_pred):
    eval_df = dict()

    eval_df["r2"] = r2_score(y_pred = y_pred, y_true = y_true).round(2)
    eval_df["mape"] = mean_absolute_percentage_error(y_pred = y_pred, y_true = y_true).round(2)
    eval_df["mae"] = mean_absolute_error(y_pred = y_pred, y_true = y_true).round(2)
    eval_df["mse"] = mean_squared_error(y_pred = y_pred, y_true = y_true).round(2)

    return pd.DataFrame([eval_df])

evaluate_model(fe.y_test, pred)


,r2,mape,mae,mse
0,-0.4,0.09,46.58,6302.95


In [15]:
evaluate_model(fe.y_train, my_model.model.predict(X_train_array))

178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 622us/step


,r2,mape,mae,mse
0,0.87,0.08,46.53,4215.38


In [16]:
test_df = fe.df.loc[fe.y_test.index, :]

In [17]:
import plotly.express as px

px.scatter(x = fe.y_test.index, y = [fe.y_test, pd.DataFrame(pred)[0]])